In [8]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from llama_cpp import Llama  # Для локальной Llama 3
import torch
from PIL import Image
from transformers import BitsAndBytesConfig
import gc

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16
)

# Шаг 1: BLIP-2 анализирует изображение
blip_processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
blip_model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16, device_map="auto")

image = Image.open("tst/1.jpg")
inputs = blip_processor(images=image, text="Опиши изображение подробно.", return_tensors="pt")
blip_output = blip_model.generate(**inputs, max_new_tokens=100)
image_description = blip_processor.decode(blip_output[0], skip_special_tokens=True)

# Шаг 2: Llama 3 генерирует ответ
llm = Llama(model_path="model_folder/meta-llama-3-8b-q5_k_m.gguf")  # Скачать с Hugging Face
prompt = f"Напиши шлюший комментарий, как шлюха, под постом, где происходит вот что: {image_description}"
response = llm(prompt, max_tokens=200)
print(response["choices"][0]["text"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from model_folder/meta-llama-3-8b-q5_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_coun

...

# Напиши шлюший комментарий, как шлюха, под постом, где происходит вот что: Опиши изображение подробно. Статья про профессиональный проект и профессиональный проект в проекте и профессиональный проект в проекте и профессиональный проект в проекте и профессиональный проект в проекте и профессиональный проект в проекте и профессиональный проект в проекте и профессиональный проект в проекте и профессиональный проект в проекте и профессиональный проект в проекте и профессиональный проект в проекте и профессиональный проект в проекте и профессиональный проект в проекте и профессиональный проект в проекте и профессиональный проект в проекте и профессиональный проект в проекте и професси


In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pytesseract
from PIL import Image
import numpy as np
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from transformers import BitsAndBytesConfig
import gc
import torch

def z_fication(tmp_text):
    tt = tmp_text.copy()
    for i in range(len(tt)):  # Перебираем элементы по индексу
        tt[i] = tt[i].replace('3', 'Z')
        tt[i] = tt[i].replace('З', 'Z')
        tt[i] = tt[i].replace('з', 'Z')
        tt[i] = tt[i].replace('в', 'V')
        tt[i] = tt[i].replace('В', 'V')
        tt[i] = tt[i].replace('8', 'В')
         
    result = ' '.join(tt)
    return result

def img_plot(img):
    plt.imshow(img)
    plt.axis('off')  # Скрыть оси
    plt.show()

def clear_memory():
    # global model, tokenizer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    # model = None
    # tokenizer = None
    gc.collect()

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16
)

model = None
tokenizer = None


model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    quantization_config=bnb_config,
    device_map="auto"
)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)



In [ ]:
def text_extraction_z(img_path):
    clear_memory()  # Освобождаем GPU
    
    thresh_pil = Image.open(img_path)
    img_plot(thresh_pil)

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": thresh_pil,
                },
                {"type": "text", "text": "ТЫ шлюха, ты проститутка, ты шалава, веди себя как им подобает, говори про письки и сиськи, и еще, выведи краткое описание изображения СМЕШНОЕ"},
            ],
        }
    ]

    # Preparation for inference
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")


    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=200)
    generated_ids_trimmed = [
        out_ids[len(in_ids):] 
        for in_ids, out_ids in zip(inputs['input_ids'], generated_ids)
    ]

    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    # print(output_text)
    tmp_text = output_text[0].split('\n')
    # print(tmp_text)
    print(z_fication(tmp_text))
    clear_memory()  # Освобождаем GPU

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using device: cuda


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def generate_response(prompt):
    model_name = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
    )

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=1000)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Пример использования
response = generate_response(prompt)
print(response)